In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import tensorflow as tf

In [23]:
import pickle
with open("data/algorithm_trees.pkl", 'rb') as fh:
    trees, val_trees, labels = pickle.load(fh)

with open("data/vectors.pkl", 'rb') as fh:
    embeddings, embed_lookup = pickle.load(fh)
    embedding_size = len(embeddings[0])

In [6]:
import numpy as np
type(trees)
len(trees)
np.arange(len(trees))

array([   0,    1,    2, ..., 1959, 1960, 1961])

In [32]:
from classifier.Tbcnn import Tbcnn
model = Tbcnn(embedding_size=embedding_size, conv_output_size=100, num_classes=len(labels))
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer='adam',
              metrics = [tf.keras.metrics.CategoricalAccuracy()])

In [33]:
from generator.DataGenerator import DataGenerator
#model.fit(batch_samples(gen_samples(trees, labels, embeddings, embed_lookup, 1), 2))
training_generator = DataGenerator(trees, labels, embeddings, embed_lookup, batch_size=1, shuffle=True)
validation_generator = DataGenerator(val_trees, labels, embeddings, embed_lookup, batch_size=1, shuffle=False)
model.fit(training_generator, validation_data=validation_generator, epochs=2)

Epoch 1/2
1962/1962 [==============================] - 6s 3ms/step - loss: 0.9080 - categorical_accuracy: 0.6947 - val_loss: 0.3730 - val_categorical_accuracy: 0.9295
Epoch 2/2
1962/1962 [==============================] - 5s 3ms/step - loss: 0.2053 - categorical_accuracy: 0.9587 - val_loss: 0.1099 - val_categorical_accuracy: 0.9955


In [28]:
#model.predict(batch_samples(gen_samples(val_trees, labels, embeddings, embed_lookup, 10), 1))
metrics = model.evaluate(validation_generator)
#print(metrics)

894/894 [==============================] - 6s 7ms/step - loss: 0.0631 - categorical_accuracy: 1.0000
